# Extraction of population counts by age and sex to construct population pyramid

## Purpose
This notebook extracts participant counts by age and sex from the baseline Our Future Health (OFH) questionnaire to construct a population pyramid.

## Outputs
- `inputs/phenotype_files/population_pyramids/pop_pyramids_ons_ofh.xlsx`, sheet `age_sex_counts_ofh_v12`.
  Table reporting participant counts by age and sex.

  
## Relationship to manuscript
Results from this notebook are used to create **Figure 1.e** (*Overview of Our Future Health data and cohort: population age–sex structure compared with the mid-2023 population estimates for Great Britain from the Office of National Statistics (ONS)*).

## Data and access data
Analyses use restricted Our Future Health data accessed within the OFH Trusted Research Environment under approved study permissions. Outputs are limited to aggregated, non-disclosive summary statistics in accordance with OFH Safe Output policies.

# Get data

In [ ]:
# Import libraries
import pandas as pd, pyspark, dxdata, dxpy, os, logging, json
import matplotlib.pyplot as plt
import numpy as np

# Configure logging
logging.basicConfig(level=logging.INFO)

# Download and load config file
try:
    dxpy.download_dxfile("file-J35yx8k2gbP4K6JQx7xfVQVx", "config.json")
    with open("config.json", "r") as f:
        config = json.load(f)
except Exception as e:
    raise

!dx download "profile_study_v3:/applets/ofh_tools/" -r 
#needed to put the option --overwrite in order to download utils, because wihtout it stops at process.py 

# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

#from ofh_tools import utils

import sys
sys.path.append("/opt/notebooks/ofh_tools")

import utils

#load dataset
dataset = utils.connect_to_dataset()

#cohort = dxdata.load_dataset(id = "project-Gzb9b712JqxqFY0jVbZb6KZk:record-GzbB5Yk2kzj312GFGkPK3p8P")

#variables of interest: sex = demog_sex_1_1 & demog_sex_2_1; age = registration_year (registration_month) - birth_year (birth_month)
field_names = [
    'registration_year',
    'registration_month',
    'birth_year',
    'birth_month',
    'demog_sex_1_1',
    'demog_sex_2_1',
    'housing_income_1_1',
    'demog_ethnicity_1_1'
]

df = dataset.retrieve_fields(names=field_names, engine=dxdata.connect())

# Convert to Pandas
pdf = df.toPandas()

# Preprocessing

## Data Cleaning and Exclusions

# Exclude invalid birth year/month
pdf = pdf[pdf['participant$birth_year'] != -999]
pdf = pdf[pdf['participant$birth_month'] != -999]

# Exclude sex values: 3 (Intersex), -3 (Prefer not to answer)
pdf = pdf[~pdf['participant$demog_sex_1_1'].isin([3, -3])]
pdf = pdf[~pdf['participant$demog_sex_2_1'].isin([3, -3])]

# Optional: Exclude certain ethnicity values
# pdf = pdf[~pdf['participant$demog_ethnicity_1_1'].isin([19, -3])]

# Optional: Exclude income responses -1 (Don't know), -3 (Prefer not to answer), and NaN
# pdf = pdf[~pdf['questionnaire$housing_income_1_1'].isin([-1, -3, np.nan])]

## Age Calculation

In [ ]:
# Calculate age in months and convert to years
pdf['age_months'] = ((pdf['participant$registration_year'] - 1) * 12 + pdf['participant$registration_month']) - ((pdf['participant$birth_year'] - 1) * 12 + pdf['participant$birth_month'])
pdf['age'] = (pdf['age_months'] / 12).round()

# Calculate age using datetime
pdf['registration_date'] = pd.to_datetime(dict(year=pdf['participant$registration_year'], month=pdf['participant$registration_month'], day=1))
pdf['birth_date'] = pd.to_datetime(dict(year=pdf['participant$birth_year'], month=pdf['participant$birth_month'], day=1))
pdf['datetime_age'] = (pdf['registration_date'] - pdf['birth_date']).dt.days / 365.25

## Sex variable transformation

pdf['sex'] = np.where(pdf['participant$demog_sex_2_1'].notna(), pdf['participant$demog_sex_2_1'], pdf['participant$demog_sex_1_1'])
pdf['sex'] = pdf['sex'].map({1: 'Male', 2: 'Female'})

## Inspect data

pdf.info()
pdf.describe()
pdf.head()

# Plotting

In [ ]:
pdf['age_group'] = pdf['datetime_age'].astype(int)
age_sex_counts = pdf.groupby(['age_group', 'sex']).size().unstack(fill_value=0)
sex_totals = age_sex_counts.sum()
age_sex_props = age_sex_counts.divide(sex_totals, axis=1)
age_sex_props['Male'] = -age_sex_props.get('Male', 0)

plt.figure(figsize=(16, 12))
plt.barh(age_sex_props.index, age_sex_props['Male'], label='Male', color='steelblue')
plt.barh(age_sex_props.index, age_sex_props['Female'], label='Female', color='salmon')
plt.xlabel('Proportion within sex')
plt.ylabel('Age')
plt.title('Population Pyramid (Proportions by Sex and Age)')
plt.legend()
plt.grid(True, axis='x')
plt.tight_layout()
plt.show()

## Inspect long data frames

age_sex_counts.info()
age_sex_counts.describe()
age_sex_counts.head()

age_sex_props.info()
age_sex_props.describe()
age_sex_props.head()

## Export data

### Population Table (Long and Wide Formats)

In [ ]:
# Long format
age_sex_counts = pdf.groupby('age_group')['sex'].value_counts().reset_index()
age_sex_counts.columns = ['Age', 'Sex', 'Count']
age_sex_counts = age_sex_counts.sort_values(by=['Age', 'Sex']).reset_index(drop=True)
age_sex_counts.info()
age_sex_counts.head()

# Long format
pdf_pop = pdf.groupby('age')['sex'].value_counts().reset_index()
pdf_pop.columns = ['Age', 'Sex', 'Count']
pdf_pop = pdf_pop.sort_values(by=['Age', 'Sex']).reset_index(drop=True)
pdf_pop.info()
pdf_pop.head()

# Wide format
table = pdf.groupby('age')['sex'].value_counts().unstack(fill_value=0)
table['Total'] = table.sum(axis=1)
print(table)

reshaped_table = table.T
reshaped_table['Total'] = reshaped_table.sum(axis=1)
reshaped_table.loc['Total'] = reshaped_table.sum(numeric_only=True)
reshaped_table.index.name = 'Sex'
reshaped_table.columns.name = 'Age'

with pd.option_context('display.max_columns', None, 'display.max_rows', None):
    print(reshaped_table)

## Export

age_sex_counts.to_csv("age_sex_counts_ofh.csv")
reshaped_table.to_csv("population_table_ofh.csv")
pdf_pop.to_csv("population_df_ofh.csv")

# Old code

# Import libraries
import pandas as pd, pyspark, dxdata, dxpy, os, logging, json
import numpy as np

# Configure logging
logging.basicConfig(level=logging.INFO)

# Download and load config file
try:
    dxpy.download_dxfile("file-J0V3q692Jqxqjp6xkYZzPXfv", "config.json")
    with open("config.json", "r") as f:
        config = json.load(f)
except Exception as e:
    raise

!dx download "profile_study_v1:/applets/ofh_tools/" -r

# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

from ofh_tools import utils

#load dataset
dataset = utils.connect_to_dataset()

cohort = dxdata.load_dataset(id = "project-Gzb9b712JqxqFY0jVbZb6KZk:record-GzbB5Yk2kzj312GFGkPK3p8P")

#variables of interest: sex = demog_sex_1_1 & demog_sex_2_1; age = registration_year (registration_month) - birth_year (birth_month)
field_names = [
    'registration_year',
    'registration_month',
    'birth_year',
    'birth_month',
    'demog_sex_1_1',
    'demog_sex_2_1',
    'housing_income_1_1',
    'demog_ethnicity_1_1'
]

df = dataset.retrieve_fields(names=field_names, engine=dxdata.connect())

#df.describe()
df.head(5)

# Convert to Pandas
pdf = df.toPandas()

pdf.info()
pdf.describe()
#pdf['participant$demog_sex_1_1'].value_counts(dropna=False)
#pdf['participant$demog_sex_2_1'].value_counts(dropna=False)
#pdf['participant$demog_ethnicity_1_1'].value_counts(dropna=False)
#pdf['participant$registration_year'].value_counts(dropna=False)
#pdf['participant$registration_month'].value_counts(dropna=False)
#pdf['participant$birth_month'].value_counts(dropna=False)

## Preprocess dataframe for analyses

### Exclusions

birth year
- exclude -999

birth month
- exclude -999

In [ ]:
pdf = pdf[pdf['participant$birth_year'] != -999]
pdf = pdf[pdf['participant$birth_month'] != -999]

sex
- exclude: 3 Intersex
- exclude: -3 Prefer not to answer

In [ ]:
pdf = pdf[~pdf['participant$demog_sex_1_1'].isin([3, -3])]
pdf = pdf[~pdf['participant$demog_sex_2_1'].isin([3, -3])]

ethnicity
- exclude: -3
- exclude: 19

In [ ]:
#pdf = pdf[~pdf['participant$demog_ethnicity_1_1'].isin([19, -3])]

income
- exclude: -1 (do not know)
- exclude: -3 (prefer not to answer)
- exclude nan

In [ ]:
#pdf = pdf[~pdf['questionnaire$housing_income_1_1'].isin([-1, -3, np.nan])]

### Transformations

Get age var: get total months, divide by 12 and round at the nearest integer (0.5 round up)

pdf['age_months'] = ((pdf['participant$registration_year'] - 1) * 12 + pdf['participant$registration_month']) - ((pdf['participant$birth_year'] - 1) * 12 + pdf['participant$birth_month'])
pdf['age'] = (pdf['age_months']/12).round()

Get age var programmatically via datetime logic

# Convert registration and birth info to datetime objects (day set to 1)
pdf['registration_date'] = pd.to_datetime(
    dict(year=pdf['participant$registration_year'],
         month=pdf['participant$registration_month'],
         day=1)
)

pdf['birth_date'] = pd.to_datetime(
    dict(year=pdf['participant$birth_year'],
         month=pdf['participant$birth_month'],
         day=1)
)

# Compute age in years, accounting for leap years
pdf['datetime_age'] = (pdf['registration_date'] - pdf['birth_date']).dt.days / 365.25

Get sex var

pdf['sex'] = np.where(pdf['participant$demog_sex_2_1'].notna(), pdf['participant$demog_sex_2_1'], pdf['participant$demog_sex_1_1'])
pdf['sex'] = pdf['sex'].map({1: 'Male', 2: 'Female'})

pdf.info()
pdf.describe()
pdf.head()

import pandas as pd
import matplotlib.pyplot as plt

# Ensure age is integer for grouping
pdf['age_group'] = pdf['datetime_age'].astype(int)

# Compute population count by age and sex
age_sex_counts = pdf.groupby(['age_group', 'sex']).size().unstack(fill_value=0)

# Convert to proportion within each sex
sex_totals = age_sex_counts.sum()
age_sex_props = age_sex_counts.divide(sex_totals, axis=1)

# Reverse male proportions for left side
age_sex_props['Male'] = -age_sex_props.get('Male', 0)

# Plot
plt.figure(figsize=(16, 12))
plt.barh(age_sex_props.index, age_sex_props['Male'], label='Male', color='steelblue')
plt.barh(age_sex_props.index, age_sex_props['Female'], label='Female', color='salmon')
plt.xlabel('Proportion within sex')
plt.ylabel('Age')
plt.title('Population Pyramid (Proportions by Sex and Age)')
plt.legend()
plt.grid(True, axis='x')
plt.tight_layout()
plt.show()

## Get data for population pyramid

# Create dataframe in long format
pdf_pop = pdf.groupby('age')['sex'].value_counts()
pdf_pop = pdf_pop.reset_index()
pdf_pop.columns = ['Age', 'Sex', 'Count']
pdf_pop = pdf_pop.sort_values(by=['Age', 'Sex']).reset_index(drop=True)
pdf_pop.info()
pdf_pop.head

# Do same thing in table format
table = pdf.groupby('age')['sex'].value_counts().unstack(fill_value=0)
table['Total'] = table.sum(axis=1)
print(table)

# Transpose table
table = pdf.groupby('age')['sex'].value_counts().unstack(fill_value=0)
reshaped_table = table.T
reshaped_table['Total'] = reshaped_table.sum(axis=1)
reshaped_table.loc['Total'] = reshaped_table.sum(numeric_only=True)
reshaped_table.index.name = 'Sex'
reshaped_table.columns.name = 'Age'

print(reshaped_table)


# Show all table
with pd.option_context('display.max_columns', None, 'display.max_rows', None):
    print(reshaped_table)

# Create csv for table
reshaped_table.to_csv("population_table_ofh.csv")

# Create csv for dataframe
pdf_pop.to_csv("population_df_ofh.csv")